In [ ]:
swid = ""
espn_s2 = ""
league_id = ""
season = 2022
n_active_players = 8
week = 1
scoring_period = 7
home_team = ""
away_team = ""
start_date = ""
end_date = ""

In [ ]:
import sys
import pandas as pd
sys.path.append('../')
from src.EspnFantasyMatchUp import EspnFantasyMatchUp
from src.utils.io_json import read_json

# Input Parameters

How to find your league's parameters:
* `swid`: If your league is public, leave blank. Otherwise: In Safari these cookies can be found by opening the Storage tab of Developer tools (developer tools can be turned on in preferences), and looking under espn.com in the Cookies folder. In Chrome, they can be found in Settings -> Cookies and other site data -> See all cookies and site data, and search for ESPN.
* `espn_s2`: If your league is public, leave blank. Otherwise, follow above instructions
* `league_id`: Go to your ESPN league page. The URL contains `leagueId=123455`. Copy the number next to the `leagueid` parameters
* `season`: Leave this to current season, i.e. 2022, unless you want to run analysis of a round of a previous season
* `n_active_players`: most leagues have 8 players playing each day. Change this accordingly.

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [ ]:
cookies = {
    "swid": swid,
    "espn_s2": espn_s2.replace("!", "%")
}
league_settings = {
    "league_id": int(league_id),
    "season": season,
    "n_active_players": n_active_players
}

In [ ]:
espn = EspnFantasyMatchUp(
    cookies, league_settings,
    week, home_team=home_team, away_team=away_team, start_date=start_date, end_date=end_date,
    stat_type_code="002022"
)

# Head to head comparison of season historic stats

In [ ]:
espn.h2h_season_stats_comparison()

# Compare Schedule (players per day)

In [ ]:
espn.compare_schedules()

# Simulate Schedule

In [ ]:
espn.simulation(use_current_score=True)

# Scenarios

Add or remove players from the roster of either the home or the away team

```
home_scn_players={
    "add": {
        <plyaer_name>: [<date1>, <date2>],
        <player_name2>: [<date1>]
    }, 
    "remove": {
        <plyaer_name3>: [<date1>, <date2>],
        <player_name4>: [<date1>]
    }, 
}
```
Similar of the away players, using the `away_scn_players` argument. 

Leave date list empyt if you want to include all dates in the schedule between `start_data` and `end_date`

In [ ]:
# scn = EspnFantasyMatchUp(
#     week, home_team=home_team, away_team=away_team, start_date=pd.to_datetime("2022-02-16"), end_date=end_date,
# #     home_scn_players={"add": {"Eric Gordon": []}, "remove": {"Nicolas Batum": []}}, 
#     stat_type_code="022022"
# )

In [ ]:
# scn.simulation(use_current_score=True)